In [641]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import requests
import xml.etree.ElementTree as ET

from scipy import interpolate

from yahoo_fin import options as op, stock_info as si


In [845]:
import datetime
import pytz


def chicago_eod_four_PM(eod, near_term=True):
    # Create a datetime object for 4 PM on November 10
    # replace 2023 with the correct year
    # 14.17 as of 11th Nov 2020
    four_pm = datetime.datetime(
        eod['year'], eod['month'], eod['day'], eod['hour'], eod['minute'], 0)  # 10th Nov 2023

    # Convert the datetime object to Eastern Time
    eastern = pytz.timezone('US/Eastern')
    four_pm_et = eastern.localize(four_pm)
    print("four_pm_et: ", four_pm_et)

    tomorrow = datetime.datetime(
        four_pm_et.year, four_pm_et.month, four_pm_et.day, tzinfo=four_pm_et.tzinfo) + datetime.timedelta(1)
    days = [four_pm_et + datetime.timedelta(index) for index in range(24, 38)]

    friday = next(day for day in days if day.weekday() == 4)
    next_term = -1 if near_term == True else 6
    days_to_expiration = (friday - four_pm_et).days + next_term
    expiration_day = datetime.datetime(
        four_pm_et.year, four_pm_et.month, four_pm_et.day,
        tzinfo=four_pm_et.tzinfo) + datetime.timedelta(days_to_expiration + 1)
    print("days to expiration: ", days_to_expiration)
    print("friday: ", friday)
    print("expiration day: ", expiration_day)
    print("expiration day - calculation day: ", expiration_day - four_pm_et)

    # minutes_to_settlement = 570 if friday.day in range(     #friday.day to expiration_day.day will cover near and next term
    #     15, 22) and friday.weekday() == 4 else 960

    minutes_to_settlement = 570 if expiration_day.day in range(
        15, 22) and friday.weekday() == 4 else 960

    minutes_to_midnight_today = (tomorrow - four_pm_et).seconds // 60
    minutes_to_expiration = days_to_expiration * 24 * 60

    print(minutes_to_midnight_today, minutes_to_settlement, minutes_to_expiration)
    time_to_expiration = (minutes_to_midnight_today +
                          minutes_to_settlement + minutes_to_expiration)

    # time_to_expiration = (minutes_to_midnight_today +
    #                       minutes_to_settlement + minutes_to_expiration) / 525600

    return (time_to_expiration, expiration_day)


yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
eod = {
    "year": yesterday.year,
    "month": yesterday.month,
    "day": yesterday.day,
    "hour": 16,
    "minute": 0,
    "second": 0
}
print("Time to expiration: ", chicago_eod_four_PM(
    eod, True)[1], chicago_eod_four_PM(eod, False)[1])


four_pm_et:  2023-11-21 16:00:00-05:00
days to expiration:  23
friday:  2023-12-15 16:00:00-05:00
expiration day:  2023-12-15 00:00:00-05:00
expiration day - calculation day:  23 days, 8:00:00
480 570 33120
four_pm_et:  2023-11-21 16:00:00-05:00
days to expiration:  30
friday:  2023-12-15 16:00:00-05:00
expiration day:  2023-12-22 00:00:00-05:00
expiration day - calculation day:  30 days, 8:00:00
480 960 43200
Time to expiration:  2023-12-15 00:00:00-05:00 2023-12-22 00:00:00-05:00


In [395]:
import json
import requests
import re


def scrape_us_treasury_yield_curve_and_transpose_it_to_minutes_scale():
    """
    Retrieves and formats the US Treasury yield curve.
    No arguments.
    """
    year = datetime.datetime.now().year
    month = datetime.datetime.now().month
    url = f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/pages/xmlview?data=daily_treasury_yield_curve&field_tdr_date_value_month={year}{month:02d}"
    response = requests.get(url)

    #print(json.dumps(response.content.decode(), indent=4))
    root = ET.fromstring(response.content)

    temp_us_treasury_dict = {}
    us_treasury_dict = {}

    for elt in root.iter():
        temp_us_treasury_dict[elt.tag[-8:]] = elt.text

    for key in temp_us_treasury_dict.keys():
        if (key.find("MONTH") + key.find("YEAR") + key.find("DATE") != -3):
            us_treasury_dict[re.sub(r'.*_', '', key)
                             ] = temp_us_treasury_dict[key]

    # Based on the average number of day in a month: 30.42
    minutes_in_a_month = 43804
    minutes_in_a_year = 525600

    # minute axis
    #x = [int(key[:-5])*minutes_in_a_month if "MONTH" in key else int(key[:-4])*minutes_in_a_year if "YEAR" in key else None for key in us_treasury_dict.keys()]
    x = [int(key[:-5])*minutes_in_a_month if "MONTH" in key else int(key[:-4]) *
         minutes_in_a_year if "YEAR" in key else None for key in us_treasury_dict.keys() if "MONTH" in key or "YEAR" in key]

    # yield axis
    y = [us_treasury_dict[key]
         for key in us_treasury_dict.keys() if "MONTH" in key or "YEAR" in key]
    print("Yield curve:", us_treasury_dict)
    print("Yield curve in minutes scale:", dict(zip(x, y)))

    return [x, y]

#r = scrape_us_treasury_yield_curve_and_transpose_it_to_minutes_scale()
# r


In [812]:
import numpy as np
from scipy.interpolate import CubicSpline
from datetime import date
import math


def cubic_spline_risk_free_rate(minutes_to_expiration):
    """
    Estimates the risk free rate (based on the US treasury yield) at a specific time to expiration
    :param <time_to_expiration>: integer ; time to expiration in day or minutes 
    :param <minute_data>: boolean ; indicates if the argument <time_to_expiration> is in minutes or days
    """

    # Extract the tenors and corresponding yields from the US Treasury yield curve
    yc = scrape_us_treasury_yield_curve_and_transpose_it_to_minutes_scale()
    tenors = yc[0]
    rates = yc[1]

    # Sort the data by tenor
    sorted_indices = np.argsort(tenors)
    # print(sorted_indices)
    #tenors_sorted = tenors[sorted_indices.tolist()]
    #rates_sorted = rates[sorted_indices.tolist()]

    tenors_sorted = [tenors[i] for i in sorted_indices]
    rates_sorted = [rates[i] for i in sorted_indices]

    # print(tenors_sorted)
    # print(rates_sorted)

    # Create a cubic spline interpolator
    cs = CubicSpline(tenors_sorted, rates_sorted,
                     extrapolate=True)  # Allow extrapolation

    # Specify expiration dates of near-term and next-term options (replace with actual dates)
    #near_term_expiration = date(2023, 12, 31)
    #next_term_expiration = date(2024, 3, 31)

    # Calculate time remaining until expiration in years
    #today = date.today()
    #time_to_near_term = (near_term_expiration - today).days / 365
    #time_to_next_term = (next_term_expiration - today).days / 365

    time_to_near_term = minutes_to_expiration[0]
    time_to_next_term = minutes_to_expiration[1]
    #time_to_near_term = 34,484
    #time_to_next_term = 44,954

    # Calculate interpolated/extrapolated yields using cubic spline
    yield_R1 = cs(time_to_near_term)
    yield_R2 = cs(time_to_next_term)
    # print(f"R1 (BEY) for {time_to_near_term} minutes: {yield_R1}")
    # print(f"R2 (BEY) for {time_to_next_term} minutes: {yield_R2}")

    # Convert BEY to APY using the correct formula
    APY_R1 = ((1 + yield_R1 / 2) ** 2) - 1
    APY_R2 = ((1 + yield_R2 / 2) ** 2) - 1

    r_near = np.log(1 + APY_R1)
    r_next = np.log(1 + APY_R2)

    # print(f"R1 (APY) for {time_to_near_term} minutes: {APY_R1}")
    # print(f"R2 (APY) for {time_to_next_term} minutes: {APY_R2}")

    # print(f"R1 (ln(APY+1)) for {time_to_near_term} minutes: {r_near}")
    # print(f"R2 (ln(APY+1)) for {time_to_next_term} minutes: {r_next}")

    # return [APY_R1, APY_R2]
    # return [r_near, r_next]
    return [yield_R1, yield_R2]


In [806]:
import numpy as np
import pandas as pd


def extrapolate_cmt(t):
    # Sort the data by maturity
    yc = scrape_us_treasury_yield_curve_and_transpose_it_to_minutes_scale()
    print(yc)
    cmt_data = pd.DataFrame({'maturity': yc[0], 'rate': yc[1]})
    cmt_data['rate'] = pd.to_numeric(cmt_data['rate'])
    cmt_data = cmt_data.sort_values(by='maturity')

    # Extract the maturities and rates
    maturities = cmt_data['maturity'].values
    rates = cmt_data['rate'].values

    # Find the indices of the two maturities that bracket t
    idx = np.searchsorted(maturities, t)

    # If t is within the range of maturities, interpolate
    if 0 < idx < len(maturities):
        CMTi = rates[idx - 1]
        CMTi1 = rates[idx]
        BEYt = min(CMTi, CMTi1) if t < maturities[idx] else max(CMTi, CMTi1)
        print("BEYt from IF: ", BEYt)
    # If t is outside the range of maturities, extrapolate
    else:
        t1, CMT1 = maturities[0], rates[0]
        tx, CMTx = maturities[1], rates[1] if rates[1] >= rates[0] else (0, 0)
        tz, CMTz = maturities[1], rates[1] if rates[1] <= rates[0] else (0, 0)

        m0_lower = (CMTx - CMT1) / (tx - t1) if tx != 0 else 0
        b_lower = CMT1 - m0_lower * t1
        m0_upper = (CMTz - CMT1) / (tz - t1) if tz != 0 else 0
        b_upper = CMT1 - m0_upper * t1

        BEYt_lower = m0_lower * t + b_lower
        BEYt_upper = m0_upper * t + b_upper
        # print(BEYt_lower, BEYt_upper)
        # print(type(BEYt_lower), type(BEYt_upper))
        #BEYt = min(BEYt_lower, BEYt_upper)[0]
        BEYt = np.minimum(BEYt_lower, BEYt_upper)[0]
        print("BEYt from Else: ", BEYt)
        print(type(BEYt))

    # Convert the BEY rate to a continuously compounded APY rate
    APYt = (1 + BEYt / 2) ** 2 - 1
    rt = np.log(1 + APYt)
    print("rt: ", rt)

    return rt


extrapolate_cmt(39990)
extrapolate_cmt(50460)


Yield curve: {'DATE': '2023-11-20T00:00:00', '1MONTH': '5.53', '2MONTH': '5.55', '3MONTH': '5.54', '4MONTH': '5.46', '6MONTH': '5.43', '1YEAR': '5.25', '2YEAR': '4.89', '3YEAR': '4.62', '5YEAR': '4.44', '7YEAR': '4.46', '10YEAR': '4.42', '20YEAR': '4.74', '30YEAR': '4.57'}
Yield curve in minutes scale: {43804: '5.53', 87608: '5.55', 131412: '5.54', 175216: '5.46', 262824: '5.43', 525600: '5.25', 1051200: '4.89', 1576800: '4.62', 2628000: '4.44', 3679200: '4.46', 5256000: '4.42', 10512000: '4.74', 15768000: '4.57'}
[[43804, 87608, 131412, 175216, 262824, 525600, 1051200, 1576800, 2628000, 3679200, 5256000, 10512000, 15768000], ['5.53', '5.55', '5.54', '5.46', '5.43', '5.25', '4.89', '4.62', '4.44', '4.46', '4.42', '4.74', '4.57']]
BEYt from Else:  5.528258606519953
<class 'numpy.float64'>
rt:  2.6510331475052857
Yield curve: {'DATE': '2023-11-20T00:00:00', '1MONTH': '5.53', '2MONTH': '5.55', '3MONTH': '5.54', '4MONTH': '5.46', '6MONTH': '5.43', '1YEAR': '5.25', '2YEAR': '4.89', '3YEAR':

2.651495722503714

In [399]:
def add_strike_diff(df):
    # Create a new column 'strike_prev' that contains the previous 'strike' values
    df['strike_prev'] = df['strike'].shift(-1)

    # Create a new column 'strike_next' that contains the next 'strike' values
    df['strike_next'] = df['strike'].shift(1)

    # Create a new column 'strike_diff' that contains the differences between 'strike_next' and 'strike_prev'
    df['strike_diff'] = (df['strike_prev'] - df['strike_next'])/2

    # For the first and last 'strike', keep the original 'strike' values
    # df.loc[0, 'strike_diff'] = df.loc[1, 'strike'] - df.loc[0, 'strike']
    df.iloc[0, df.columns.get_loc('strike_diff')] = df.iloc[1, df.columns.get_loc('strike')] - \
        df.iloc[0, df.columns.get_loc('strike')]

    df.iloc[-1, df.columns.get_loc('strike_diff')] = df.iloc[-1, df.columns.get_loc('strike')] -\
        df.iloc[-2, df.columns.get_loc('strike')]
    # df.loc[df.index[-1], 'strike_diff'] = df.loc[df.index[-1],
    #                                              'strike'] - df.loc[df.index[-2], 'strike']
    return df


In [401]:
def remove_strikes(df, option_type):
    # Create new columns that contain the 'bid' values of the preceding and following rows
    df['bid_prev1'] = df['bid'].shift(1)
    df['bid_next1'] = df['bid'].shift(-1)

    # For 'Call' options, find the first index where 'bid' and 'bid_next1' are 0
    if option_type == 'Call':
        try:
            index_to_drop = df[(df['bid'] == 0) & (
                df['bid_next1'] == 0)].index[0]
            # Drop all rows from 'index_to_drop' to the end of the DataFrame
            df = df.loc[:index_to_drop-1]
        except IndexError:
            pass

    # For 'Put' options, find the first index where 'bid' and 'bid_prev1' are 0
    elif option_type == 'Put':
        try:
            index_to_drop = df[(df['bid'] == 0) & (
                df['bid_prev1'] == 0)].index[0]
            # Drop all rows from the start of the DataFrame to 'index_to_drop'
            df = df.loc[index_to_drop+1:]
        except IndexError:
            pass

    # Drop the 'bid_prev1' and 'bid_next1' columns
    df = df.drop(columns=['bid_prev1', 'bid_next1'])

    # remove any row that has zeor bid
    df = df[df['bid'] != 0]

    #display(HTML(df.to_html(index=False, border=0)))

    return df


In [438]:
def forward_and_atm(min_diff_strike, df_calls, df_puts, activation, near_term):
    """
    Calculate forward VIX by extraploating minimum strike price...
    """
    df_calls_copy = df_calls.copy().set_index('strike', inplace=False)
    df_puts_copy = df_puts.copy().set_index('strike', inplace=False)

    forward = min_diff_strike + \
        activation * (df_calls.loc[df_calls['strike'] == min_diff_strike, 'mid-quote'].values[0] -
                      df_puts.loc[df_puts['strike'] == min_diff_strike, 'mid-quote'].values[0])

    print("Forward: ", forward)
    common_strikes = df_calls_copy.index.intersection(df_puts_copy.index)

    lower_strikes = common_strikes[common_strikes < forward]
    atm_strike = lower_strikes.max()

    #atm_strike = df_calls[df_calls['strike'] < f_near]['strike'].max()

    row_calls = df_calls.loc[df_calls['strike']
                             == atm_strike].reset_index(drop=True)
    row_puts = df_puts.loc[df_puts['strike']
                           == atm_strike].reset_index(drop=True)
    average_values = (row_calls[['mid-quote', 'bid', 'ask', 'px_last']] +
                      row_puts[['mid-quote', 'bid', 'ask', 'px_last']]) / 2
    average_values['strike'] = atm_strike
    average_values['option_type'] = 'ATM Avg Put/Call'
    atm_df = average_values[['strike', 'bid', 'ask',
                             'option_type', 'px_last', 'mid-quote']]

    display(HTML(atm_df.to_html(index=False, border=0)))
    return forward, atm_df, atm_strike


In [439]:
def find_min_diff_strike(df_calls, df_puts):
    # Set the strike price as the index in both dataframes
    df_calls_copy = df_calls.copy()
    df_puts_copy = df_puts.copy()

    df_calls_copy.set_index('strike', inplace=True)
    df_puts_copy.set_index('strike', inplace=True)

    # Calculate the difference between the mid-quote values

    mid_quote_diff = (df_calls_copy['mid-quote'] -
                      df_puts_copy['mid-quote']).abs()

    # Find the strike price with the minimum difference
    min_diff_strike = mid_quote_diff.idxmin()

    print("min_diff_strike: ", min_diff_strike)
    return min_diff_strike


In [527]:
def find_min_diff_strike_new(df_calls, df_puts, activation, near_term):
    # Set the strike price as the index in both dataframes
    df_calls_local = df_calls.copy()
    df_puts_local = df_puts.copy()

    df_calls_local['strike-copy'] = df_calls_local['strike']
    df_puts_local['strike-copy'] = df_puts_local['strike']

    df_calls_local.set_index('strike', inplace=True)
    df_puts_local.set_index('strike', inplace=True)

    # Find common strikes
    common_strikes = df_calls_local.index.intersection(df_puts_local.index)

    # Filter dataframes to only include common strikes
    df_calls_local = df_calls_local.loc[common_strikes]
    df_puts_local = df_puts_local.loc[common_strikes]

    # Filter out rows where mid-quote is zero in both dataframes
    non_zero_mask = ~((df_calls_local['mid-quote'] == 0)
                      & (df_puts_local['mid-quote'] == 0))
    df_calls_local = df_calls_local[non_zero_mask]
    df_puts_local = df_puts_local[non_zero_mask]

    # Calculate the difference between the mid-quote values
    mid_quote_diff = (
        df_calls_local['mid-quote'] - df_puts_local['mid-quote']).abs()

    # Find the strike price with the minimum difference
    min_diff_strike = mid_quote_diff.idxmin()
    print("min_diff_strike: ", min_diff_strike)

    #######################################################################################
    forward = min_diff_strike + \
        activation * (df_calls.loc[df_calls['strike'] == min_diff_strike, 'mid-quote'].values[0] -
                      df_puts.loc[df_puts['strike'] == min_diff_strike, 'mid-quote'].values[0])

    print("Forward: ", forward)

    atm_strike_another = df_calls_local.loc[df_calls_local['strike-copy']
                                            < forward, 'strike-copy'].max()
    print("atm_strike-another: ", atm_strike_another)

    atm_strike = df_calls_local[df_calls_local['strike-copy']
                                < forward]['strike-copy'].max()

    print("atm_strike: ", atm_strike)
    ###display(HTML(df_calls_local.to_html(index=False, border=0)))
    ###display(HTML(df_puts_local.to_html(index=False, border=0)))

    row_calls = df_calls.loc[df_calls['strike']
                             == atm_strike].reset_index(drop=True)
    row_puts = df_puts.loc[df_puts['strike']
                           == atm_strike].reset_index(drop=True)
    average_values = (row_calls[['mid-quote', 'bid', 'ask', 'px_last']] +
                      row_puts[['mid-quote', 'bid', 'ask', 'px_last']]) / 2
    average_values['strike'] = atm_strike
    average_values['option_type'] = 'ATM Avg Put/Call'
    atm_df = average_values[['strike', 'bid', 'ask',
                             'option_type', 'px_last', 'mid-quote']]

    display(HTML(atm_df.to_html(index=False, border=0)))
    return forward, atm_df, atm_strike
    ##############################################################################################

    # return min_diff_strike


In [467]:
def preapare_data(df):
    # Split the dataframe into calls and puts
    df_calls = df[df['option_type'] == 'C'].copy()
    df_puts = df[df['option_type'] == 'P'].copy()

    df_puts.loc[:, 'mid-quote'] = (df_puts['bid'] + df_puts['ask']) / 2
    df_calls.loc[:, 'mid-quote'] = (df_calls['bid'] + df_calls['ask']) / 2

    return df_calls, df_puts


In [717]:
import pandas as pd
from IPython.display import display, HTML


def chain_of_responsibility(df, r, t, near_term=True):

    df = df.fillna(0)

    df_calls, df_puts = preapare_data(df)

    # Remove the strikes with no bid price
    df_puts, df_calls = [remove_strikes(
        df_puts, 'Put'), remove_strikes(df_calls, 'Call')]

    # if near_term is False:
    #     display(HTML(df_puts.to_html(index=False, border=0)))
    #     display(HTML(df_calls.to_html(index=False, border=0)))

    # Find the min strike difference
    ######min_diff_strike = find_min_diff_strike_new(df_calls, df_puts, near_term)

    ###########################
    activation = np.exp(r*t)
    forward, df_atm, atm_strike = find_min_diff_strike_new(
        df_calls, df_puts, activation, near_term)
    ##########################

    # Calculate Forwrard
    # activation = np.exp(r*t)
    # forward, df_atm, atm_strike = forward_and_atm(
    #     min_diff_strike, df_calls.copy(), df_puts.copy(), activation, near_term)

    # activation = np.exp(r*t)
    # forward, df_atm, atm_strike = calculate_forward(
    #     min_diff_strike, df_calls, df_puts, activation, near_term)

    # Filter the dataframe to include only OTM strike
    df_puts = df_puts[df_puts['strike'] < atm_strike]
    df_calls = df_calls[df_calls['strike'] > atm_strike]

    # Remove the strikes with no bid price
    df_puts, df_calls = [remove_strikes(
        df_puts, 'Put'), remove_strikes(df_calls, 'Call')]

    # Combine data frames
    df_otm = pd.concat([df_puts, df_atm, df_calls], ignore_index=True)
    df_otm = df_otm.sort_values('strike')

    # Add adjescent strike difference
    df_otm = add_strike_diff(df_otm)

    # Contribution per Strike
    df_otm['strike_squared'] = df_otm['strike'] ** 2
    df_otm['strike_contribution'] = (
        df_otm['strike_diff'] * df_otm['mid-quote'] * activation)/df_otm['strike_squared']

    #display(HTML(df_otm.to_html(index=False, border=0)))
    total_contribution = 2 * df_otm['strike_contribution'].sum()/t
    #print("Total contribution", total_contribution)
    decay = ((forward/atm_strike - 1) ** 2)/t

    sigma_squared = (total_contribution - decay)
    #print("Sigma squared near", sigma_squared_near)
    return sigma_squared


In [829]:
import pandas as pd
from datetime import date
from thetadata import ThetaClient, DateRange, StockReqType


def end_of_day() -> pd.DataFrame:
    client = ThetaClient(username="sauravlefty@gmail.com",
                         passwd="Vix!2345")  # No credentials required for free access

    # Make any requests for data inside this block. Requests made outside this block won't run.
    with client.connect():
        out = client.get_hist_stock(
            req=StockReqType.EOD,  # End of day data
            root="AAPL",
            date_range=DateRange(date(2022, 11, 14), date(2022, 11, 18))
        )
    # We are out of the client.connect() block, so we can no longer make requests.
    return out


data = end_of_day()
print(type(data))
print(data.to_string())


If you require API support, feel free to join our discord server! http://discord.thetadata.us
[11-21-2023 18:39:29] INFO: Starting Theta Terminal v0.9.12 Revision D...
[11-21-2023 18:39:29] INFO: Working Dir: C:\Users\USER\ThetaData\ThetaTerminal
[11-21-2023 18:39:30] INFO: [MDDS] Attempting login as sauravlefty@gmail.com
[11-21-2023 18:39:31] INFO: [FPSS] Attempting login as sauravlefty@gmail.com
[11-21-2023 18:39:31] INFO: [MDDS] CONNECTED: [nj-a.thetadata.us:12000], Bundle: FREE, Permissions: End of Day(EOD)
<class 'pandas.core.frame.DataFrame'>
   DataType.OPEN  DataType.HIGH  DataType.LOW  DataType.CLOSE  DataType.VOLUME  DataType.COUNT DataType.DATE
0         149.30       166.6500      135.3900         148.770         80889202          576303    2022-11-14
1         149.51       153.5900      140.9150         149.700         96309849          703679    2022-11-15
2         150.25       155.0800      138.9800         148.980         71463364          472255    2022-11-16
3        

In [848]:
import datetime
# Time to expiration
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
eod = {
    "year": yesterday.year,
    "month": yesterday.month,
    "day": yesterday.day,
    "hour": 16,
    "minute": 15,
    "second": 0
}

calc_day = str(yesterday.day)+yesterday.strftime('%b')
print(calc_day)

n_t1, n_t2 = chicago_eod_four_PM(
    eod, near_term=True)[0], chicago_eod_four_PM(eod, near_term=False)[0]
print("n_t1 and n_t2", n_t1, n_t2, )
# extrapolated rates to expiration
r_near, r_next = np.array(cubic_spline_risk_free_rate(
    [n_t1, n_t2]))/100
print("r_near and r_next", r_near, r_next, )

t_near = n_t1/525600
t_next = n_t2/525600

df_near = pd.read_json(
    f'c:/Python311/Lib/site-packages/yahoo_fin/dataset/{calc_day}/SPXW US -15-12-2023_data.json')
df_next = pd.read_json(
    f'c:/Python311/Lib/site-packages/yahoo_fin/dataset/{calc_day}/SPXW US -22-12-2023_data.json')

# df_near = pd.read_csv(
#     'c:/Python311/Lib/site-packages/yahoo_fin/dataset/15thNov/SPXW US -15-12-2023_data.csv')
# df_next = pd.read_csv(
#     'c:/Python311/Lib/site-packages/yahoo_fin/dataset/15thNov/SPX US -22-12-2023_data.csv')


sigma_squared_near, sigma_squared_next = (chain_of_responsibility(df_near, r_near, t_near),
                                          chain_of_responsibility(df_next, r_next, t_next, False))

print("Sigma squared near", sigma_squared_near)
print("Sigma squared next", sigma_squared_next)

n_30 = 43200
n_365 = 525600
n_y_m = n_365/n_30

tnear_sig_squared = sigma_squared_near * t_near
tnext_sig_squared = sigma_squared_next * t_next


near_dev = tnear_sig_squared * ((n_t2 - n_30)/(n_t2 - n_t1))
next_dev = tnext_sig_squared * ((n_30 - n_t1)/(n_t2 - n_t1))
tot_dev = near_dev + next_dev
vix = np.sqrt(tot_dev * n_y_m) * 100
print(vix)


21Nov
four_pm_et:  2023-11-21 16:15:00-05:00
days to expiration:  23
friday:  2023-12-15 16:15:00-05:00
expiration day:  2023-12-15 00:00:00-05:00
expiration day - calculation day:  23 days, 7:45:00
465 570 33120
four_pm_et:  2023-11-21 16:15:00-05:00
days to expiration:  30
friday:  2023-12-15 16:15:00-05:00
expiration day:  2023-12-22 00:00:00-05:00
expiration day - calculation day:  30 days, 7:45:00
465 960 43200
n_t1 and n_t2 34155 44625
Yield curve: {'DATE': '2023-11-21T00:00:00', '1MONTH': '5.52', '2MONTH': '5.54', '3MONTH': '5.53', '4MONTH': '5.45', '6MONTH': '5.42', '1YEAR': '5.24', '2YEAR': '4.86', '3YEAR': '4.60', '5YEAR': '4.41', '7YEAR': '4.44', '10YEAR': '4.41', '20YEAR': '4.75', '30YEAR': '4.57'}
Yield curve in minutes scale: {43804: '5.52', 87608: '5.54', 131412: '5.53', 175216: '5.45', 262824: '5.42', 525600: '5.24', 1051200: '4.86', 1576800: '4.60', 2628000: '4.41', 3679200: '4.44', 5256000: '4.41', 10512000: '4.75', 15768000: '4.57'}
r_near and r_next 0.05520540402698

strike,bid,ask,option_type,px_last,mid-quote
4545,50.05,50.55,ATM Avg Put/Call,50.4,50.3


min_diff_strike:  4550
Forward:  4552.511744327069
atm_strike-another:  4550
atm_strike:  4550


strike,bid,ask,option_type,px_last,mid-quote
4550,56.75,57.55,ATM Avg Put/Call,58.8,57.15


Sigma squared near 0.01693911796753623
Sigma squared next 0.017098976377498944
13.069726303486387
